# The Drunkard's Walk Again

## Introduction

I would like to spend some time on a simple random walk on the non-negative integers `0 ... k` called "the drunkard's walk"  A *lot* of intuition can be had by studying the this system. This note is a quicker way to derive some of the observations found in ["A Slightly Unfair Game"](https://win-vector.com/2023/10/30/a-slightly-unfair-game/).

I have found this differs from many treatments of Markov chains in that we are analyzing and absorbing chain (instead of a "regular" or "ergodic" one, see Kemeny, Snell, *Finite Markov Chains*, 2nd Edition, 1976, Springer Verlag) and we are using bespoke recurrence arguments instead of the usual move to linear algebra.


## The drunkard's walk

Pick an integer `k > 0`, the states of our random process are the integers `0` through `k`. The integers `0` and `k` are both "stop conditions" called "stop zero" and "stop non-zero". For an integer `i` strictly larger than `0` and strictly less than `k`, our random process is: pick the next integer to be either `i-1` or `i+1` with equal probability. We stop the process with `i` is either `0` or `k`. This is called a "random walk" and is often known as "the drunkard's walk". Some variations have "stay probabilities", but we will not need these here.

What isn't emphasized enough is: a lot can be quickly seen for this walk, without using complicated tools or arguments. In particular the following can all be shown for this system.

  1) The probability of a walk started in state-`i` "stopping non-zero" (first reaching `k`, with no prior visits to `0`) is `i/k`.
  2) The expected time of a walk started in state-`i` to hit either of the stopping conditions (reaching `0` or `k` for the first time) is `i * (k - i)`.


## Deriving the probability of "stopping non-zero"

Let `prob_pos(k, i)` denote the probability that the drunkard's walk started at integer `i` (`0 <= i <= k`) stops at `k` before ever visiting `0`.

For any `i` with `0 < i < k` we can expand `prob_pos(k, i)` by one walk step to get: `prob_pos(k, i) = (1/2) prob_pos(k, i-1) + (1/2) prob_pos(k, i+1)`. It is just a matter of algebra to check that `prob_pos(k, i) = i/k` obeys this recurrence, and has the desired values for the boundary `i = 0, k`.

As a warm up we confirm the claim.


In [1]:
import sympy

i, k = sympy.symbols("i k")

check = sympy.expand(
    ((1/2) * (i-1)/k + (1/2) * (i+1)/k) 
    - i/k)
assert check == 0

print(check)


0


## Deriving the "expected time to stop"

Let `e_time(k, i)` denote the expected number of steps the drunkard's walk started at integer `i` (`0 <= i <= k`) takes to reach *either* of `0` or `k` for the first time. We are not yet specifying which one is first reached.

For any `i` with `0 < i < k` we can expand `e_time(k, i)` by one walk step to get: `e_time(k, i) = 1 + (1/2) e_time(k, i-1) + (1/2) e_time(k, i+1)`. It is again, a matter of  algebra to check that `e_time(k, i) = i * (k-i)` obeys this recurrence, and has the desired values on the boundary.

Let's confirm this claim.


In [2]:
check = sympy.expand(
    (1 + (1/2) * (i-1) * (k-(i-1)) + (1/2) * (i+1) * (k-(i+1)))
    - i * (k-i))
assert check == 0

print(check)


0


The above directly gives us the "expected time to step distance `d` is around `d**2` steps" observation without the usual appeal to linear of expectation applied to independent variances. Note: we the expected stopping time to hit `0` is going to often be very different than the time to hit `k` (and both roughly proportional to the square of how far the specified target is from the start position).


# Defining Markov chains

For our a given run of our random process call the states <code>s<sub>0</sub></code>, <code>s<sub>1</sub></code>, ... . Where:

  * All the <code>s<sub>j</sub></code> are integers in the range `0` through `k`.
  * If <code>s<sub>j</sub></code> isn't `0` or `k`, then <code>|s<sub>i</sub> - s<sub>j+1</sub>| = 1</code> (the process has not stopped)
  * If <code>s<sub>j</sub></code> is `0` or `k`, then <code>s<sub>j+1</sub> = s<sub>j</sub>| = 1</code> (the chain has "stopped").

Let <code>S(i, k)</code> denote the set of all possible sequences <code>s</code> of integers obeying the above rules where:

  * The first state <code>s<sub>1</sub> = i</code>.
  * The "probability" of <code>s</code> is equal to:
     * <code>P[s] = 1</code>, if <code>s<sub>j+1</sub> = s<sub>j</sub></code> for all <code>j</code>. Call such <code>s</code> "stopped."
     * <code>P[s] = 0</code>, if <code>s<sub>j+1</sub> != s<sub>j</sub></code> for all <code>j</code>. Call such <code>s</code> "not stopped."
     * <code>P[s] = 1/2<sup>j-1</code></code> where <code>j</code> is the highest index such that <code>s<sub>j+1</sub> != s<sub>j</sub></code>. Call such <code>s</code> "stopped."

On can check that the <code>P[s] >= 0</code> for all <code>s</code> in <code>S(i, k)</code>, and <code>sum<sub>s in S(i, k)</sub> = 1</code>.

<code>P[]</code> is computing probability over the spaces of all possible sequences <code>s<sub>i</sub></code> generated by picking increasing or decreasing our state (for non stopped sequences) with 50/50 probability. It is a subtle point: probabilities are a distributional property of the family of all Markov chains, not of any one Markov chain.

The above random process <code>S(i, k)</code> has "the Markov property": history becomes irrelevant. That is:

<code>
<pre>
   P[s<sub>i</sub> = v | s<sub>1</sub>, ..., s<sub>i-1</sub>] = P[s<sub>i</sub> = v | s<sub>i-1</sub>]
</pre>
</code>.

For every stopped <code>s</code> we have either <code>s</code> becomes stuck on (or stopped at) <code>0</code> or stuck on (or stopped at) <code>k</code>. Call these "stopped at <code>0</code>" and "stopped at </code>k</code>", respectively.

We often use the Markov property in analysis as follows: we can always pretend we are at the first move in a sequence! <code>P[s<sub>i+1</sub> = v<sub>2</sub> | s<sub>i</sub> = v<sub>1</sub>] = P[s<sub>2</sub> = v<sub>2</sub> | s<sub>1</sub> = v<sub>1</sub>]</code> always. We also use the fact that the non-stopped sequences have probability measure zero, so can be ignored in probability arguments.



